In [1]:
import os 
from glob import glob 
from os import path

from itertools import groupby, filterfalse

import pvl
import zlib

import importlib
import os
from glob import glob

from abc import ABC, abstractmethod

from pfeffernusse.examples import get_path

os.environ['SPICE_DATA'] = '/data/spice'

def get_metakernels(missions=set(), years=set(), versions=set()):
    """
    Spicerack in a nutshell. This is mostly filtering, might be worth using Pandas here?
    
    Super beta, proof of concept code slapped together at a Starbucks.
    """
    spice_dir = os.environ.get("SPICE_DATA")
    if spice_dir is None:
        raise Exception("$SPICE_DATA not set")
    
    if isinstance(missions, str):
        missions = {missions}
        
    if isinstance(years, str) or isinstance(years, int):
        years = {str(years)}
    else:
        years = {str(year) for year in years}
    
    avail = {
        'count': 0,
        'data': []
    }
    
    mission_dirs = list(filter(os.path.isdir, glob(os.path.join(spice_dir, '*'))))
    
    for md in mission_dirs:
        # Assuming spice root has the same name as the original on NAIF website"
        mission = os.path.basename(md).split('-')[0]
        if missions and mission not in missions:
            continue
        
        metakernel_keys = ['mission', 'year', 'version', 'path']
        
        # recursive glob to make metakernel search more robust to 
        # subtle directory structure differences 
        metakernel_paths = sorted(glob(os.path.join(md, '**','*.tm'), recursive=True))
        
        metakernels = [dict(zip(metakernel_keys, [mission]+path.splitext(path.basename(k))[0].split('_')[1:3] + [k])) for k in metakernel_paths]
        
        # naive filter, do we really need anything else?
        if years:
            metakernels = list(filter(lambda x:x['year'] in years, metakernels))
        if versions:
            if versions == 'latest':  
                latest = []
                # Panda's groupby is overrated 
                for k, g in groupby(metakernels, lambda x:x['year']):
                    items = list(g)
                    latest.append(max(items, key=lambda x:x['version']))
                metakernels = latest
            else:        
                metakernels = list(filter(lambda x:x['version'] in versions, metakernels))
            
        avail['data'].extend(metakernels)
    
    avail['count'] = len(avail['data'])
    if not avail:
        avail = {
            'count' : 0,
            'data' : 'ERROR: NONE OF {} ARE VALID MISSIONS'.format(missions)
        }
        
    return avail


    

class Base(ABC):
    """
    Probably worth splitting these into mixins if we can 
    Example: Different base for line scanner vs framer 
    """
    def __init__(self, label, *args, **kwargs):
        self.label = label
    
    @property 
    def lines(self):
        return self.label['LINES']

    @property 
    def samples(self):
        return self.label['SAMPLES']
    
    @property
    def target_name(self):
        return self.label['TARGET_NAME']
    
    @property
    def et(self):
        sclock = self.label['SPACECRAFT_CLOCK_START_COUNT']
        exposure_duration = self.label['EXPOSURE_DURATION'].value
        exposure_duration = exposure_duration * 0.001  # Scale to seconds

        # Get the instrument id, and, since this is a framer, set the time to the middle of the exposure
        start_et = spice.scs2e(self.spacecraft_id, sclock)
        start_et += (exposure_duration / 2.0)
        end_et = spice.scs2e(self.spacecraft_id, self.label['SPACECRAFT_CLOCK_STOP_COUNT']) + (exposure_duration / 2.0)
        et = (start_et + end_et)/2
        return et
    
    @property
    def del_et(self):
        sclock = self.label['SPACECRAFT_CLOCK_START_COUNT']
        exposure_duration = self.label['EXPOSURE_DURATION'].value
        exposure_duration = exposure_duration * 0.001  # Scale to seconds

        # Get the instrument id, and, since this is a framer, set the time to the middle of the exposure
        start_et = spice.scs2e(spacecraft_id, sclock)
        start_et += (exposure_duration / 2.0)
        end_et = spice.scs2e(spacecraft_id, self.label['SPACECRAFT_CLOCK_STOP_COUNT']) + (exposure_duration / 2.0)
        return end_et - start_et

    @property
    def start_et(self):
        sclock = self.label['SPACECRAFT_CLOCK_START_COUNT']
        exposure_duration = self.label['EXPOSURE_DURATION'].value
        exposure_duration = exposure_duration * 0.001  # Scale to seconds

        # Get the instrument id, and, since this is a framer, set the time to the middle of the exposure
        return spice.scs2e(spacecraft_id, sclock)
 
    @property
    def spacecraft_name(self):
        return self.label['MISSION_NAME']
    
    @property
    def instrument_id(self):
        id_lookup = {
            'MDIS-NAC':'MSGR_MDIS_NAC',
            'MERCURY DUAL IMAGING SYSTEM NARROW ANGLE CAMERA':'MSGR_MDIS_NAC',
            'MERCURY DUAL IMAGING SYSTEM WIDE ANGLE CAMERA':'MSGR_MDIS_WAC'
        }
        return self.label['INSTRUMENT_ID']
    
    @property
    def ikid(self):
        return spice.bods2c(self.instrument_id)

    @property
    def spacecraft_id(self):
        return spice.bods2c(self.spacecraft_name)

    @property 
    def focal2pixel_lines(self):
        return list(spice.gdpool('INS{}_TRANSX'.format(self.ikid), 0, 3))
    
    @property 
    def focal2pixels_samples(self):
        list(spice.gdpool('INS{}_TRANSX'.format(self.ikid), 0, 3))
        
    @property 
    def focal_length(self):
        return float(spice.gdpool('INS{}_FOCAL_LENGTH'.format(self.ikid), 0, 1)[0])
            
    @property 
    def focal_length_epsilon(self):
        return float(spice.gdpool('INS{}_FL_UNCERTAINTY'.format(self.ikid), 0, 1)[0])

    @property
    def starting_detector_line(self):
        return 0
    
    @property
    def starting_detector_sample(self):
        return 0
    
    @property 
    def semimajor(self):
        rad = spice.bodvrd(self.label['TARGET_NAME'], 'RADII', 3)
        return rad[1][1]

    @property
    def semiminor(self):
        rad = spice.bodvrd(self.label['TARGET_NAME'], 'RADII', 3)
        return rad[1][0]

    @property
    def reference_frame(self):
        return 'IAU_{}'.format(self.label['TARGET_NAME'])
    
    @property
    def sun_position(self):
        sun_state, _ = spice.spkezr("SUN",
                                     self.et,
                                     self.reference_frame,
                                     'NONE',
                                     self.label['TARGET_NAME'])

        return sun_state[:4]

    @property
    def sun_velocity(self):
        sun_state, lt = spice.spkezr("SUN",
                                     self.et,
                                     self.reference_frame,
                                     'NONE',
                                     self.label['TARGET_NAME'])

        return sun_state[4:7]
    
    @property
    def sensor_velocity(self):
        v_state, _ = spice.spkezr(self.target_name,
                                           self.et,
                                           self.reference_frame,
                                           'None',
                                           self.label['TARGET_NAME'])
        return vstate[3:6]
    @property
    def sensor_position(self):
        loc, _ = spice.spkpos(self.target_name, 
                              self.et, 
                              self.reference_frame, 
                              'None', 
                              self.spacecraft_name)
        return loc[:4]
    

# Personally not a fan of haivng mixins that only serve one purpose, I would prefer simply returning 
# an array in one case and a tuple of arrays in the other, but did this as it fits the current 
# model and worth seeing the consequences of going this route 
class TransverseDistortion(ABC):
    @property
    def odtx(self):
        return spice.gdpool('INS{}_OD_T_X'.format(self.ikid),0, 10)
    
    @property 
    def odty(self):
        return spice.gdpool('INS{}_OD_T_Y'.format(self.ikid), 0, 10)

class RadialDistortion(ABC):
    @property
    def odr(self):
        return spice.gdpool('INS{}_OD_K'.format(self.ikid),0, 3)
        

In [2]:
from glob import glob
import os

import pvl
import spiceypy as spice
import numpy as np

class Messenger(Base, TransverseDistortion):
    @property
    def instrument_id(self):
        id_lookup = {
            'MDIS-NAC':'MSGR_MDIS_NAC',
            'MERCURY DUAL IMAGING SYSTEM NARROW ANGLE CAMERA':'MSGR_MDIS_NAC',
            'MERCURY DUAL IMAGING SYSTEM WIDE ANGLE CAMERA':'MSGR_MDIS_WAC'
        }
        return id_lookup[self.label['INSTRUMENT_ID']]
        
    @property
    def focal_length(self):
        """
        """
        coeffs = spice.gdpool('INS{}_FL_TEMP_COEFFS '.format(self.ikid), 0, 5)

        # reverse coeffs, mdis coeffs are listed a_0, a_1, a_2 ... a_n where
        # numpy wants them a_n, a_n-1, a_n-2 ... a_0
        f_t = np.poly1d(coeffs[::-1])

        # eval at the focal_plane_tempature
        return f_t(self.label['FOCAL_PLANE_TEMPERATURE'].value)

class Cassini(Base):
    """
    So Cassini doesn't have a distortion model, does this make the distortion model mixin
    more justified? Or could we just override and return none?
    
    Also, smalls doesn't seem to have cassini spice so this one isn't being used atm. 
    """

    @property
    def instrument_id(self):
        instrument_names = {
            "ISSNA" : "CASSINI_ISS_NAC",
            "ISSWA" : "CASSINI_ISS_WAC"
        }
        return self.label['INSTRUMENT_ID']
    
class Mro(Base, RadialDistortion):
    """
    Most of MRO's uniqueness comes from CTX as a line scanner, Does it make sense to split them?
    i.e. should drivers be Instrument or Mission specific? And is there enough common denominators between
    line scanners that there should there be a line scanner mixin? 
    """
    
    @property
    def lines(self):
        return self.label['IMAGE']['LINES']
    
    @property
    def samples(self):
        return self.label['IMAGE']['SAMPLES']
    
    @property
    def instrument_id(self):
        # no HIRISE or MARCI becuase I can't be bothered for now 
        instrument_names = {
            "CTX" : "MRO_CTX"
        }
        return instrument_names[self.label['INSTRUMENT_ID']]
    
    @property
    def spacecraft_name(self):
        """
        Spice wants MRO but labels define MARS_RECONNAISSANCE_ORBITER 
        """
        return 'MRO'
    
    @property
    def et(self):
        sclock = self.label['SPACECRAFT_CLOCK_START_COUNT']
        et = spice.scs2e(self.spacecraft_id, sclock)
        return et
    
    @property
    def starting_detector_line(self):
        return 1
    
    @property
    def starting_detector_sample(self):
        return 0
    
    @property
    def center_ephemeris_time(self):
        half_lines = self.lines / 2
        return self.et + half_lines * self.line_scan_rate
        
    @property
    def dt_ephemeris(self):
        return 80 * self.line_scan_rate  # This is every 300 lines
        
    @property
    def dt_quaternion(self):
        return self.dt_ephemeris

    @property 
    def number_of_ephemerides(self):
        # Determine how many ephemeris points to compute
        n_ephemeris = int(self.scan_duration / self.dt_ephemeris)
        if n_ephemeris % 2 == 0:
            n_ephemeris += 1
        return n_ephemeris
        
    @property 
    def scan_duration(self):
        return self.line_scan_rate * self.lines

    @property
    def line_scan_rate(self):
        return self.label['LINE_EXPOSURE_DURATION'][0]  * 0.001  # Scale to seconds
    
    @property
    def t0_ephemeris(self):
        return self.et - self.center_ephemeris_time
    
    @property
    def sensor_position(self):
        n_ephemeris = self.number_of_ephemerides
        current_et = self.et
        
        eph = np.empty((n_ephemeris, 3))
        eph_rates = np.empty(eph.shape)

        for i in range(n_ephemeris):
            loc_direct, _ = spice.spkpos(self.target_name, current_et, self.reference_frame, 'NONE', self.spacecraft_name)
            state, _ = spice.spkezr(self.target_name, current_et, self.reference_frame, 'NONE', self.spacecraft_name)
            eph[i] = loc_direct
            eph_rates[i] = state[3:]
            current_et += self.dt_quaternion # Increment the time by the number of lines being stepped
        eph *= -1000 # Reverse to be from body center and convert to meters
        eph_rates *= -1000 # Same, reverse and convert
        return eph


In [20]:
def create_isd(d):
    """
    super minimal example of how ISDs can be loaded through a single function that 
    grabs what it needs from drivers 
    """
    # Unfortunately ISD is tightly coupled to the way Drivers are composed because it needs to now the available driver types
    # and how they affect available attributes. The note on the mixins is related 
    get_distortion = lambda d:  d.odr if isinstance(d, RadialDistortion) else {'x': d.odtx, 'y': d.odty} if isinstance(d, TransverseDistortion) else None
    
    return {
    'ET': d.et,
    'FOCAL_LENGTH' : d.focal_length,
    'OPTICAL_DISTORTION': get_distortion(d),
    'SUN_POSITOIN': d.sun_position,
    'SEMIMAJOR' : d.semimajor,
    'SENSOR_POSITION': d.sensor_position
    }


In [18]:
# lol 
latest_mess = get_metakernels(missions='mess', versions='latest', years=2013)

# Should Drivers be responsible for loading/unloading kernels? 
# I imagine not since the kernel picking proccess wont be so simple 
# as done here
spice.furnsh(latest_mess['data'][0]['path'])
d = Messenger(label=pvl.load(os.path.join(get_path('labels'), 'messenger.lbl')))
isd = create_isd(d)
spice.unload(latest_mess['data'][0]['path'])
isd

{'ET': 418855170.5006496,
 'FOCAL_LENGTH': 549.2347965210602,
 'OPTICAL_DISTORTION': {'x': array([ 0.00000000e+00,  1.00185427e+00,  0.00000000e+00,  0.00000000e+00,
         -5.09444047e-04,  0.00000000e+00,  1.00401047e-05,  0.00000000e+00,
          1.00401047e-05,  0.00000000e+00]),
  'y': array([0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 9.06001059e-04,
         0.00000000e+00, 3.57484263e-04, 0.00000000e+00, 1.00401047e-05,
         0.00000000e+00, 1.00401047e-05])},
 'SUN_POSITOIN': array([-3.16407024e+07, -6.06380937e+07, -3.87311046e+04, -3.81911239e+01]),
 'SEMIMAJOR': 2439.4,
 'SENSOR_POSITION': array([-1728.37368696,  2088.42341073, -2082.86052983])}

In [19]:
ctx_label = pvl.load(os.path.join(get_path('labels'), 'mro.lbl'))
latest_mro = get_metakernels(missions='mro', versions='latest',  years=ctx_label['START_TIME'].year)

# Should Drivers be responsible for loading/unloading kernels? 
spice.furnsh(latest_mro['data'][0]['path'])

d = Mro(label=ctx_label)
isd = create_isd(d)
spice.unload(latest_mro['data'][0]['path'])
isd

{'ET': 504962414.05132544,
 'FOCAL_LENGTH': 352.9271664,
 'OPTICAL_DISTORTION': array([-7.34339259e-03,  2.83758786e-05,  1.28419891e-08]),
 'SUN_POSITOIN': array([ 2.94824781e+07, -2.22404582e+08,  1.05537374e+08, -1.57427160e+04]),
 'SEMIMAJOR': 3396.19,
 'SENSOR_POSITION': array([[ 2882280.01735562, -2032514.35810954,  -957269.61505613],
        [ 2882358.36952956, -2032644.12186327,  -956774.4882776 ],
        [ 2882436.66214734, -2032773.84726679,  -956279.34246041],
        [ 2882514.89522319, -2032903.53429542,  -955784.17761447],
        [ 2882593.06877135, -2033033.18292449,  -955288.9937497 ],
        [ 2882671.18276167, -2033162.79319222,  -954793.79087603],
        [ 2882749.23719358, -2033292.36509492,  -954298.56900337],
        [ 2882827.23211092, -2033421.89856595,  -953803.32814164],
        [ 2882905.16746875, -2033551.39366453,  -953308.06830075],
        [ 2882983.04328131, -2033680.85036599,  -952812.78949064],
        [ 2883060.85956284, -2033810.26864563,  -95231

In [ ]:
get_metakernels(missions='mro', years={2016, 2015}, versions='latest')